# Set up

In [ ]:
! pip  install pymongo[srv,tls]
! pip install dnspython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 25.3 MB/s eta 0:00:00


In [ ]:
from pymongo.mongo_client import MongoClient
from datetime import date
from contextlib import nullcontext

uri = "mongodb+srv://yanaput:<password>@workshop-bakery.qtngygm.mongodb.net/?retryWrites=true&w=majority&appName=Workshop-Bakery"

client = MongoClient(uri)
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

db = client.Bakery



In [ ]:
#@title Cake Info

Name = "exampleCake" #@param {type:"string"}
ShortDescription = "example" #@param {type:"string"}
Description = "example" #@param {type:"string"}
Image = "exampleURL" #@param {type:"string"}
Ingredient = "exampleIngre0, exampleIngre1, exampleIngre2, exampleIngre3" #@param {type:"string"}
Recipe = "example text"#@param {type:"string"}
Stock =  33 #@param {type:"slider", min:1, max:80, step:1}
Price = 10 #@param {type: "number"}



In [ ]:
cake = {  'name' : Name ,
          'shortDescription' : ShortDescription,
          'description' : Description,
          'image' : Image,
          'ingredients' : Ingredient.split(","),
          'recipe' : Recipe,
          'stock' : Stock,
          'price' : Price
       }

if len(list(db.Cakes.find({'name' : cake['name']}))) > 0 :
  print("The cake already existed replacing instead")
  print(db.Cakes.replace_one({'name': cake['name']}, cake))
else :
  print(db.Cakes.insert_one(cake))


In [ ]:
#@title Customer Info

#@markdown Name
Name = "Mark" #@param {type:"string"}
LastName = "Kuprasertwong" #@param {type:"string"}

#@markdown Address
Address = "example" #@param {type:"string"}
Country = "Thailand" #@param {type:"string"}
City = "example" #@param {type:"string"}
Province = "example" #@param {type :"string"}
PostalCode = "73170" #@param {type:"string"}

#@markdown Contact information
Email = "example@email.com" #@param {type:"string"}
Phone = "0850000000" #@param {type:"string"}


In [ ]:
customerInfo = {
    'name' : Name,
    'lastName' : LastName,
    'address' : Address,
    'country' : Country,
    'city' : City,
    'province' : Province,
    'postalCode' : PostalCode,
    'email' : Email,
    'phone' : Phone }

if len(list(db.addressbook.find({'name' : customerInfo['name'],
                                'lastName' : customerInfo['lastName']}))) > 0 :
  print("The info already existed replacing instead")
  print(db.addressbook.replace_one({'name' : customerInfo['name'],
                                'lastName' : customerInfo['lastName']}, customerInfo))
else :
  print("the info not existed")
  print(db.addressbook.insert_one(customerInfo))

the info not existed
InsertOneResult(ObjectId('6648b5b407e347e5fd7dba24'), acknowledged=True)


In [ ]:
#@title Order information

CustomerName = "Mark" #@param {type:"string"}
LastName = "Kuprasertwong" #@param {type:"string"}
Cake = "Cheese Cake"#@param {type:"string"}
Quantity = 2 #@param {type:"integer"}
OrderDate = '2024-05-16' #@param {type:"date"}
RetrieveDate = '2024-05-16' #@param {type:"date"}
RetrieveOption = "Pickup" #@param ["Pickup", "Delivery"]


In [ ]:
orderedCake = db.Cakes.find_one({'name': Cake})

order = {
    'customerName' : CustomerName,
    'lastName' : LastName,
    'cake' : Cake,
    'quantity' : Quantity,
    'subTotal' : orderedCake['price'] * Quantity,
    'orderDate' : OrderDate ,
    'retrieveDate' : RetrieveDate,
    'retrieveOption' : RetrieveOption
}



if db.Cakes.find_one({'name': order['cake']}) is not None :
  updatedCake = db.Cakes.find_one_and_update({'name': order['cake']},
                      {'$inc': {'stock': -Quantity if orderedCake['stock'] > 0 and orderedCake['stock'] > Quantity else 0 }},
                      return_document=True)
  if orderedCake['stock'] < 0 :
     print(f"{order['cake']} is out of stock")
  elif orderedCake['stock'] < Quantity :
    print(f"{order['cake']} quantity is not enough")
  else:
    print(db.order.insert_one(order))
    print("Stock updated:", updatedCake)
else :
  print(f"error no {order['cake']} in the shop")



InsertOneResult(ObjectId('6648b4df07e347e5fd7dba22'), acknowledged=True)
Stock updated: {'_id': ObjectId('64759e4c3beb40594cb859ed'), 'name': 'Cheese Cake', 'shortDescription': 'Cheesecake is a sweet dessert consisting of one or more layers. The main, and thickest, layer consists of a mixture of a soft, fresh cheese (typically cottage cheese, cream cheese or ricotta), eggs, and sugar. ', 'description': 'Cheesecake is a sweet dessert consisting of one or more layers. The main, and thickest, layer consists of a mixture of a soft, fresh cheese (typically cottage cheese, cream cheese or ricotta), eggs, and sugar. If there is a bottom layer, it most often consists of a crust or base made from crushed cookies (or digestive biscuits), graham crackers, pastry, or sometimes sponge cake.[1] Cheesecake may be baked or unbaked (and is usually refrigerated).', 'image': 'https://sallysbakingaddiction.com/wp-content/uploads/2018/05/perfect-cheesecake-recipe.jpg', 'ingredients': ['graham cracker crumb

In [ ]:
#@title Search

#seach cake info
SearchCake = "Chocolate Cake" #@param {type:"string"}

#search order by cake name
SearchCakeOrder = "Chocolate Cake" #@param {type : "string"}

#search customer information
SearchCustomerInfo = "exampleName exampleLastName" #@param {type : "string"}

#search order by customer order
SearchCustomerOrder = "Mark Kuprasertwong" #@param {type : "string"}

#<--------------------------------------------------------------------------->
#Search cake by name in database
if SearchCake is not None and SearchCake != "":
  result = db.Cakes.find_one({'name' : SearchCake})
  if result is not None:
    print("Cake info : ")
    print(result)
  else:
    print(f"no {SearchCake} cake in database")

#<--------------------------------------------------------------------------->
#Search order by cake in database
if SearchCakeOrder is not None and SearchCakeOrder != "":
  result = db.order.find({'cake' : SearchCakeOrder})
  if result is not None:
    print("Cakes in orders :")
    for item in list(result):
      print(item)
  else:
    print(f"no {SearchCakeOrder} cake in order")

#<--------------------------------------------------------------------------->
#Search customer info by [name lastname] in database
if SearchCustomerInfo is not None and SearchCustomerInfo != "":
  customerNameLastname = SearchCustomerInfo.split()
  if len(customerNameLastname) > 1 :
    result = db.addressbook.find_one({'name' : customerNameLastname[0],
                                'lastName' : customerNameLastname[1]})
    if result is not None:
      print("Cutomer info : ")
      print(result)
    else:
      print(f"no {SearchCustomerInfo} customer in database")
  else:
    print("Customer info's format is incorrect")

#<--------------------------------------------------------------------------->
#Search order by [name lastname] in database
if SearchCustomerOrder is not None and SearchCustomerOrder != "":
  customerNameLastname = SearchCustomerOrder.split()
  if len(customerNameLastname) > 1 :
    result = db.order.find({'customerName' : customerNameLastname[0],
                                'lastName' : customerNameLastname[1]})
    if result is not None:
      print(f"Customer [{SearchCustomerOrder}]'s order :")
      for item in list(result):
        print(item)
    else:
      print(f"no [{SearchCustomerOrder}]'s order in database")
  else:
    print("Customer info's format is incorrect")

Cake info : 
{'_id': ObjectId('64759d513beb40594cb859e9'), 'name': 'Chocolate Cake', 'shortDescription': 'Chocolate cake is a cake flavored with melted chocolate, cocoa powder, or sometimes both.', 'description': 'Chocolate cake is made with chocolate; it can be made with other ingredients, as well. These ingredients include fudge, vanilla creme, and other sweeteners. The history of chocolate cake goes back to 1764, when Dr. James Baker discovered how to make chocolate by grinding cocoa beans between two massive circular millstones.', 'image': 'https://addapinch.com/wp-content/uploads/2020/04/chocolate-cake-DSC_1768.jpg', 'ingredients': ['flour', 'sugar', 'cocoa powder'], 'recipe': 'Preheat your oven to the specified temperature and prepare a greased cake pan. Mix the dry and wet ingredients separately, then combine and pour the batter into the pan. Bake for the recommended time, let it cool, and optionally frost or decorate as desired.', 'stock': 13, 'price': 20}
Cakes in orders :
{'_